<a href="https://colab.research.google.com/github/SumeetChougule/PM-HR/blob/main/notebooks/CNN_trial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## git clone

In [1]:
!git clone https://github.com/DifferentiableUniverseInitiative/flowpm.git

Cloning into 'flowpm'...
remote: Enumerating objects: 3470, done.
remote: Counting objects: 100% (1315/1315), done.
remote: Compressing objects: 100% (462/462), done.
remote: Total 3470 (delta 856), reused 1230 (delta 833), pack-reused 2155
Receiving objects: 100% (3470/3470), 65.54 MiB | 36.10 MiB/s, done.
Resolving deltas: 100% (2285/2285), done.


In [2]:
!pip install git+https://github.com/DifferentiableUniverseInitiative/flowpm.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/DifferentiableUniverseInitiative/flowpm.git to /tmp/pip-req-build-1gj205mf
  Running command git clone -q https://github.com/DifferentiableUniverseInitiative/flowpm.git /tmp/pip-req-build-1gj205mf
     |████████████████████████████████| 385 kB 8.2 MB/s 
     |████████████████████████████████| 221 kB 49.3 MB/s 
     |████████████████████████████████| 1.1 MB 15.3 MB/s 
  Created wheel for flowpm: filename=flowpm-0.1.2.dev252+g8f9415b-py3-none-any.whl size=45196 sha256=070af65333095dc04b57337512e9a8d4e0b10990d52d4e141f9fffed7ba09443
  Stored in directory: /tmp/pip-ephem-wheel-cache-riqg78n5/wheels/62/64/5f/3dd430bb0fd93140aa11dda1ffbb7a9c3cffb9a3743c997423
  Created wheel for bigfile: filename=bigfile-0.1.51-cp37-cp37m-linux_x86_64.whl size=568161 sha256=00d5c71db8901e490d1d490adca138d4efbfa79c34675712b25218c2797f14a9
  Stored in directory: /root/.cache/pip/wheel

## Modules

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras.layers as tfl
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
from tensorflow.keras import datasets, layers, models, losses

import tensorflow_probability as tfp
tfd = tfp.distributions

In [4]:
import flowpm
from astropy.cosmology import Planck15
from flowpm import linear_field, lpt_init, nbody, cic_paint, cic_readout
from flowpm.utils import r2c3d, c2r3d

from scipy.interpolate import InterpolatedUnivariateSpline as iuspline


# Input data from PM






In [5]:
bs, nc = 100, 32
nsteps = 5
a0, af, nsteps = 0.1, 1.0,  nsteps
stages = np.linspace(a0, af, nsteps, endpoint=True)
donbody = False
dnoise = 1. #0.1

klin, plin = np.loadtxt('flowpm/flowpm/data/Planck15_a1p00.txt').T
ipklin = iuspline(klin, plin)

In [6]:
@tf.function
def pm(linear):
    print("PM graph")
    cosmo = flowpm.cosmology.Planck15()
    state = lpt_init(cosmo, linear, a=a0, order=2)
    final_state = nbody(cosmo, state,  stages, nc)
    tfinal_field = cic_paint(tf.zeros_like(linear), final_state[0])
    return final_state, tfinal_field

In [7]:
ic = linear_field(nc, bs, ipklin, name='pm').numpy()
state, fin = pm(tf.constant(ic))
data = fin + np.random.normal(0, dnoise, nc**3).reshape(fin.shape)

data = data.numpy().astype(np.float32)

PM graph


In [14]:
data.shape

(1, 32, 32, 32)

In [8]:
ip_cnn = tf.expand_dims(data,-1)
ip = ip_cnn.shape
ip[1:]

TensorShape([32, 32, 32, 1])

### Helper functions

In [32]:
def cic_readout_features(mesh, part, name="CiCReadout"):
  """
  Reads out particles from mesh.
  Parameters: 
  ----------- 
  mesh: tensor (batch_size, nc, nc, nc, T)
      Input 4D mesh tensor with last axis of T features
  
  part: tensor (batch_size, npart, 3)
      List of 3D particle coordinates, assumed to be in mesh units if
  boxsize is None
  
  Return:
  -------
  value: tensor (batch_size, npart) 
      Value of the field sampled at the particle locations
  """
  with tf.name_scope("CiCReadoutFeatures"):
    mesh = tf.convert_to_tensor(mesh, name="mesh")
    part = tf.convert_to_tensor(part, name="part")

    shape = tf.shape(mesh)
    batch_size, nx, ny, nz = shape[0], shape[1], shape[2], shape[3]
    nc = [nx, ny, nz]

    # Flatten part if it's not already done                                                                                                                                                                                                                                                                                   
    if len(part.shape) > 3:
      part = tf.reshape(part, (batch_size, -1, 3))

    # Extract the indices of all the mesh points affected by each particles                                                                                                                                                                                                                                                   
    part = tf.expand_dims(part, 2)
    floor = tf.floor(part)
    connection = tf.expand_dims(
        tf.constant([[[0, 0, 0], [1., 0, 0], [0., 1, 0], [0., 0, 1],
                      [1., 1, 0], [1., 0, 1], [0., 1, 1], [1., 1, 1]]]), 0)

    neighboor_coords = tf.add(floor, connection)
    kernel = 1. - tf.abs(part - neighboor_coords)
    kernel = kernel[..., 0] * kernel[..., 1] * kernel[..., 2]

    neighboor_coords = tf.cast(neighboor_coords, tf.int32)
    neighboor_coords = tf.math.mod(neighboor_coords, nc)

    meshvals = tf.gather_nd(mesh, neighboor_coords, batch_dims=1)
    weightedvals = tf.multiply(meshvals, tf.expand_dims(kernel, -1))
    value = tf.reduce_sum(weightedvals, axis=-2)
    return value

## CNN architecture

CNN $→$ CIC_readout $→$ Flatten $→ MLP $

In [28]:
def cnn_model(input_shape):
    model = tf.keras.Sequential([
        tfl.InputLayer(input_shape= input_shape),
        tfl.Conv3D(filters = 2, kernel_size= 3, strides=(1,1,1), padding='same', activation= 'relu',data_format='channels_last'),
        tfl.Conv3D(filters = 4, kernel_size= 3, strides=(1,1,1), padding='same', activation= 'relu',data_format='channels_last')
        ])
    return model

cnn = cnn_model(ip[1:])

In [29]:
cnn_op=cnn(data)

In [30]:
cnn_op.shape

TensorShape([1, 32, 32, 32, 4])

In [31]:
cnn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_11 (Conv3D)          (None, 32, 32, 32, 2)     56        
                                                                 
 conv3d_12 (Conv3D)          (None, 32, 32, 32, 4)     220       
                                                                 
Total params: 276
Trainable params: 276
Non-trainable params: 0
_________________________________________________________________


Interpolation of CNN output

In [34]:
pos = state[0].numpy()

In [55]:
p_pos = cic_readout_features( cnn_op , pos)
MLP_ip = tfl.Flatten()(p_pos)
MLP_ip.shape[1]

131072

In [48]:
nn_ip=tf.squeeze(MLP_ip)

In [56]:
def MLP():
  model = tf.keras.Sequential([
      tfl.InputLayer(input_shape = MLP_ip.shape[1]),
      tfl.Dense(64, activation = 'relu'),
      tfl.Dense(32, activation = 'relu'),
      tfl.Dense( 2, activation = 'linear')
  ])
  return model 

f_op = MLP()

In [57]:
f_op.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                8388672   
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 2)                 66        
                                                                 
Total params: 8,390,818
Trainable params: 8,390,818
Non-trainable params: 0
_________________________________________________________________


In [58]:
f_op(MLP_ip)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.11627301,  0.13111848]], dtype=float32)>